#Install Dependencies

In [ ]:
!git clone https://github.com/meituan/YOLOv6
%cd YOLOv6
!pip install -r requirements.txt

# Download Data


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL

from google.colab import files
from google.colab import drive
import pathlib

In [ ]:
def upload_drive():
  drive.mount('/content/drive')

upload_drive()

Mounted at /content/drive


In [ ]:
!mkdir yolov6data
!cp '/content/drive/My Drive/AARD/Data/all_images.v1i.mt-yolov6.zip' ./yolov6data


In [ ]:
!cd /content && unzip /content/YOLOv6/yolov6data/all_images.v1i.mt-yolov6.zip
!cp '/content/drive/My Drive/AARD/Data/data.yaml' /content/

# Begin Custom Training



In [ ]:
# begin training
!python tools/train.py --batch 32 --conf configs/yolov6s.py --eval-interval 5 --epochs 150 --img-size 1024 --data /content/data.yaml --device 0

Using 1 GPU for training... 
training args are: Namespace(batch_size=32, check_images=False, check_labels=False, conf_file='configs/yolov6s.py', data_path='/content/data.yaml', device='0', dist_url='env://', epochs=150, eval_final_only=False, eval_interval=5, gpu_count=0, heavy_eval_range=50, img_size=1024, local_rank=-1, name='exp', output_dir='./runs/train', rank=-1, resume=False, save_dir='runs/train/exp3', workers=8, world_size=1, write_trainbatch_tb=False)

Train: Final numbers of valid images: 73/ labels: 73. 
0.0s for dataset initialization.
Convert to COCO format
100% 17/17 [00:00<00:00, 24146.01it/s]
Convert to COCO format finished. Resutls saved in /content/annotations/instances_valid.json
Val: Final numbers of valid images: 17/ labels: 17. 
0.0s for dataset initialization.
Model: Model(
  (backbone): EfficientRep(
    (stem): RepVGGBlock(
      (nonlinearity): ReLU(inplace=True)
      (se): Identity()
      (rbr_dense): Sequential(
        (conv): Conv2d(3, 32, kernel_size=(

In [ ]:
!cp runs/train/exp3/weights/best_ckpt.pt "/content/drive/My Drive/AARD/yolov6s-all-best.pt"

# Evaluation

In [ ]:
# Run evaluation
!python tools/eval.py --data /content/data.yaml --img-size 1024 --weights runs/train/exp3/weights/best_ckpt.pt --device 0 


Namespace(batch_size=32, conf_thres=0.001, data='/content/data.yaml', device='0', half=False, img_size=1024, iou_thres=0.65, name='exp', save_dir='runs/val/', task='val', weights='runs/train/exp3/weights/best_ckpt.pt')
Loading checkpoint from runs/train/exp3/weights/best_ckpt.pt

Fusing model...
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Switch model to deploy modality.
Model Summary: Params: 17.19M, Gflops: 112.83
Val: Checking formats of labels with 2 process(es): 
17 label(s) found, 0 label(s) missing, 0 label(s) empty, 0 invalid label files: 100% 17/17 [00:00<00:00, 1980.70it/s]
Convert to COCO format
100% 17/17 [00:00<00:00, 43851.89it/s]
Convert to COCO format finished. Resutls saved in /content/annotations/instance

# Inference


In [ ]:
# infer on all images in our /test directory
!python tools/infer.py --yaml /content/data.yaml --img-size 1024 --weights runs/train/exp3/weights/best_ckpt.pt --source /content/images/test/ --device 0


In [ ]:
# display test inference result images
import glob
from IPython.display import Image, display

i = 0
limit = 17 # max images to print
for imageName in glob.glob('./runs/inference/exp/*.jpg'): #assuming JPG
    if i < limit:
      display(Image(filename=imageName))
      print("\n\n")
    i = i + 1